![logo](geemap_logo.png)

In [ ]:
#Install required libraries
%pip install geemap
%pip install geedim

In [ ]:
#Load libraries
import os
import ee
import geemap

In [ ]:
#Create an interactive GEE map instance to view in notebook
Map = geemap.Map()

#Load study area from earth engine asset
study_area = ee.FeatureCollection('projects/ee-harryseely/assets/NB_boundary_epsg-4326')

#Display the view to the center of the screen and scale the view
Map.centerObject(study_area, 7)

#Add study area to map
styling = {'color': "red", 'fillColor': '00000000'}
Map.addLayer(study_area.style(**styling), None, 'Study Area')

#View map
Map

In [ ]:
# A function that scales and masks Landsat 8 (C2) surface reflectance images.
# Source: https://gis.stackexchange.com/questions/425159/how-to-make-a-cloud-free-composite-for-landsat-8-collection-2-surface-reflectanc
def prepSrL8(image):
  # Develop masks for unwanted pixels (fill, cloud, cloud shadow).
  qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
  saturationMask = image.select('QA_RADSAT').eq(0)

  # Apply the scaling factors to the appropriate bands.
  def getFactorImg(factorNames):
    factorList = image.toDictionary().select(factorNames).values()
    return ee.Image.constant(factorList)

  scaleImg = getFactorImg([
    'REFLECTANCE_MULT_BAND_.|TEMPERATURE_MULT_BAND_ST_B10'])
  offsetImg = getFactorImg([
    'REFLECTANCE_ADD_BAND_.|TEMPERATURE_ADD_BAND_ST_B10'])
  scaled = image.select('SR_B.|ST_B10').multiply(scaleImg).add(offsetImg)

  # Replace original bands with scaled bands and apply masks.
  return image.addBands(scaled, None, True) \
    .updateMask(qaMask).updateMask(saturationMask)

In [ ]:
# Landsat 8 Collection 2 surface reflectance images of interest.
ls8_comp_per_year = ee.ImageCollection([]); # Create an empty image collection to store composites.

# Define the time range.
startYear = 2016
endYear = 2021
years = list(range(startYear, endYear, 1))

#Loop through years to get a summer cloud free composite of study area
for year in years:

  #Set date range
  startDate = ee.Date.fromYMD(year, 6, 1)
  endDate = ee.Date.fromYMD(year, 8, 31)

  #Generate composite from collection
  composite = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(study_area) \
    .filterDate(startDate, endDate) \
    .filter(ee.Filter.calendarRange(5, 8, 'month')) \
    .map(prepSrL8) \
    .select('SR_B.|ST_B10') \
    .median() \
    .clip(study_area)

  #Add year to metadata
  composite = composite.set('year', year)

  #Append year composite to collection
  ls8_comp_per_year = ls8_comp_per_year.merge(composite); # Add the composite to the collection.

In [ ]:
#Select images for 3 years
composite2016 = ls8_comp_per_year.filterMetadata('year', 'equals', 2016).first()
composite2018 = ls8_comp_per_year.filterMetadata('year', 'equals', 2018).first()
composite2020 = ls8_comp_per_year.filterMetadata('year', 'equals', 2020).first()

#Plot composite images for three years
Map.addLayer(composite2016, {'bands':['SR_B4',  'SR_B3',  'SR_B2'], 'min':0, 'max':0.3}, 'Landsat-8 2016 Composite')
Map.addLayer(composite2018, {'bands':['SR_B4',  'SR_B3',  'SR_B2'], 'min':0, 'max':0.3}, 'Landsat-8 2018 Composite')
Map.addLayer(composite2020, {'bands':['SR_B4',  'SR_B3',  'SR_B2'], 'min':0, 'max':0.3}, 'Landsat-8 2020 Composite')

In [ ]:
#Check band names
composite2016.bandNames()

In [ ]:
#Create a grid to use for tiling the imagery (due to GEE memory limits...)
grid = geemap.fishnet(study_area, rows=2, cols=2)
Map.addLayer(grid, {}, 'Grids')

In [ ]:
#Get study area geometry
roi = study_area.geometry()

#Loop through all years and download imagery in tiles
for year in years:

    print(f"Downloading tiles for {year} composite.")

    out_dir = fr"F:\NB_Satellite_Imagery\NB_Landsat_8_Cloud_Free_Comp\ls8_{year}_comp"

    # Check if the directory exists
    if not os.path.exists(out_dir):
        # If it doesn't exist, create it
        os.makedirs(out_dir)

    #Extract comp for year from collection
    comp_img = ls8_comp_per_year.filterMetadata('year', 'equals', year).first()

    #Clip image to specific bounding geometry of study area
    comp_img = comp_img.clip(roi).unmask()

    #Download image by tile
    geemap.download_ee_image_tiles(
        comp_img, features=grid, out_dir=out_dir, prefix=f"l8_comp_{year}",  scale=30, crs='EPSG:2953')